# Deeplearning with Transformer Architectures

(Explanation of method here)

In [ ]:
# Import all relevant modules
import utils

# Read file and preprocess
df = utils.get_dataset("Data/AV_trial.csv", preprocessing_params={"stop words","alphanumeric","stem"}) #TODO change when data is released properly


['must', 'admit', 'sat', 'watch', 'pleasantli', 'surpris', 'funnier', 'imagin', 'howev', 'expect', 'low', 'realli', 'mean', 'film', 'anyth', 'averag', 'pretti', 'good', 'amount', 'laugh', 'noth', 'memor', 'end', 'seem', 'much', 'happi', 'end', 'obvious', 'done', 'open', 'film', 'mainstream', 'crowd', 'plot', 'decent', 'enough', 'obvious', 'written', 'put', 'charact', 'unlik', 'situat', 'least', 'littl', 'pace', 'pretti', 'good', 'film', 'realli', 'bore']
['caught', 'plane', 'one', 'movi', 'basic', 'one', 'made', 'feel', 'least', 'amount', 'strong', 'repuls', 'avail', 'choic', 'endur', '90', 'minut', 'pretti', 'scatter', 'enjoy', 'funni', 'thing', 'scene', 'dojo', 'bad', 'mostli', 'humor', 'realli', 'work', 'least', 'though', 'tucker', 'seem', 'grow', 'irrit', 'time', 'goe', 'charact', 'seem', 'gotten', 'somewhat', 'incompet', 'first', 'film', 'seen', 'second', 'good', 'cop', 'unorthodox', 'necessarili', 'popular', 'come', 'inept', 'sure', 'anyon', 'subject', 'sing', 'either']
['town', 